# Computational Mathematics
## An Introduction to Numerical Analysis and Scientific Computing with Python
### By Dimitrios Mitsotakis

# Chapter 5: Roots of Equations 

Very often, we need to solve an equation of the form

$$ f(x) = 0 $$

or, in other words, find the values of $x$ that satisfy the equation $f(x)=0$. If the equation is simple, we can solve this problem without the use of computers. However, when the equation is complicated or nonlinear, we usually cannot find an analytical solution.

We will focus on numerical methods for approximating the root of a general equation $f(x)=0$. Specifically, we will study the Bisection, Newton-Raphson, and Secant methods. We will also explore combinations of these methods. Additionally, we will examine the convergence of the algorithms and learn how to implement and use them in Python.

Let's start with the Bisection method.

## Bisection Method

The Bisection method is based on Bolzano's theorem of Calculus (intermediate value theorem):

**Theorem**: 


If a function $f(x)$ satisfies the following conditions:

- $f$ is continuous in $[a,b]$
- $f(a) \cdot f(b) < 0$

then there exists at least one solution $x^\ast\in (a,b)$ of the equation $f(x)=0$ 
This means that the function changes sign in the interval $[a,b]$ and $\text{sign}(f(a)) \not= \text{sign}(f(b))$.

The **Bisection method** is used to find a very small interval $[a,b]$ that contains the root of the equation $f(x)=0$ and the approximate solution is defined to be the midpoint of that interval $(a+b)/2$ 

### Bisection algorithm

Given a bracket $[a,b]$ and a function $f(x)$ 

1. Set a tolerance `TOL` for accuracy
2. Initialize the interval $[a,b]$
3. Set $k=0$
3. Check if $f(a)\cdot f(b)<0$ (if not then quit)
4. while `|b-a|>TOL` do
   - Set `x_k = (a+b)/2` (the new approximation)
   - Check in which interval $[a,x_k]$ or $[x_k,b]$ the function changes sign
   - if $f(k_x)\cdot f(a)<0$ then
       - $b=x_k$
   - else if $f(x_k)\cdot f(b)<0$ then
       - $a=x_k$
   - else
       - Break the loop because $f(x_k)=0$
   - end if
   - $k=k+1$
5. end while
6. Return the approximation of the root $x^\ast$


And here is a python implementation:

In [1]:
def bisection(f, a, b, tol = 1.e-6):
    iteration = 0 #initialize counter iteration
    if (f(a) * f(b) < 0.0): # check if there is a root
        while ((b-a) > tol): # check if the end-points converge
            iteration = iteration + 1
            x = (a + b)/2
            if (f(a) * f(x) < 0.0):
                b = x
            elif (f(x) * f(b) < 0.0):
                a = x
            else:
                break
            print(iteration, x)
    else:
        print('failure') 
    return x 
# returns the midpoint of the final interval

Let's try our method now for the equation
$$ \ln x + x = 0 $$
in the inteval $[0.1,1]$

In [2]:
import numpy as np

def f(x):
    y = np.log(x) + x
    return y

a = 0.1
b = 1.0
tol = 1.e-4
x = bisection(f, a, b, tol)
print('The aproximate solution is: ', x)
print('And the error is: ', f(x))

1 0.55
2 0.775
3 0.6625000000000001
4 0.6062500000000001
5 0.578125
6 0.5640625
7 0.57109375
8 0.567578125
9 0.5658203125000001
10 0.5666992187500001
11 0.567138671875
12 0.5673583984375
13 0.56724853515625
14 0.567193603515625
The aproximate solution is:  0.567193603515625
And the error is:  0.0001390223881425623


- Evaluating the value $f(x^\ast)$ we observe that the approximate solution satisfies the equation $f(x)=0$ with 4 decimal digitis correct. This is because we chose `tol = 1.e-4`. Try using smaller values of the variable `tol`.
- We also observe that to achieve the requested accuracy the method required 14 iterations

### Experimental convergence rate

In order to estimate the convergence rate, we compute the errors for three subsequent iterations, let's say $e_1, e_2, e_3$. Then we assume that

$$|e_2|=C|e_1|^r$$
$$|e_3|=C|e_2|^r$$

Dividing the previous equations we have

$$\frac{|e_2|}{|e_3|}=\left(\frac{|e_1|}{|e_2|}\right)^r$$

We then we solve for $n$ to obtain the formula

$$r = \frac{\log\frac{|e_2|}{|e_3|}}{\log\frac{|e_1|}{|e_2|}}$$

We modify the `bisection` function in order to compute the convergence rates numerically, and we try using the same problem as before.

In [3]:
def bisection_rates(f, a, b, tol = 1.e-6):
    iteration = 0
    if (f(a) * f(b) < 0.0):
        e1 = abs(b-a) #initialize e1 arbitrarily
        e2 = e1*2 #initialize e2  arbitrarily
        e3 = e1 #initialize e3 arbitrarily
        while ((b-a)>tol):
            e1 = e2
            e2 = e3
            iteration = iteration + 1
            x = (a + b)/2
            if (f(a) * f(x) < 0.0):
                b = x
            elif (f(x) * f(b) < 0.0):
                a = x
            else:
                break
            e3 = np.abs(b-a)
            rate = np.log(e2/e3)/np.log(e1/e2)
            print('iteration = ', iteration, 'rate =', rate)
    else:
         print('failure')
     
    return x 

In [4]:
def f(x):
    y = np.log(x) + x
    return y

a = 0.1
b = 1.0
tol = 1.e-4

x = bisection_rates(f, a, b, tol)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

iteration =  1 rate = 1.0000000000000002
iteration =  2 rate = 0.9999999999999997
iteration =  3 rate = 0.9999999999999993
iteration =  4 rate = 1.0000000000000007
iteration =  5 rate = 1.0000000000000029
iteration =  6 rate = 0.9999999999999971
iteration =  7 rate = 1.0000000000000115
iteration =  8 rate = 0.9999999999999657
iteration =  9 rate = 1.0000000000000682
iteration =  10 rate = 0.9999999999999545
iteration =  11 rate = 0.9999999999998177
iteration =  12 rate = 1.0000000000001823
iteration =  13 rate = 1.0000000000007292
iteration =  14 rate = 0.9999999999992709
The approximate solution x is:  0.567193603515625
And the value f(x) is:  0.0001390223881425623


Verify theoretically that for the previous example we need 14 iterations.

In the next lesson we continue with more sophisticated numerical methods for obtaining faster and more accurate approximations.

## Fixed Point Methods 

 - Given an interval, bisection is guaranteed to converge to a root
 - However bisection uses almost no information about $f(x)$ beyond its sign at a point
 
**Basic Idea**: Every equation of the form $f(x)=0$ can be written equivalentrly in the form $x=g(x)$ in many different ways

**Definition:** A point $x^\ast$ in the domain of the function $g$ is called **fixed point** if $x^\ast=g(x^\ast)$

### Construction of the iterative method

- We need to construct an algorithm for the computation of the fixed points of $g$
- But first, given an equation $f(x)=0$ we need to define the function $g(x)$
- There is **not** a unique way to define the function $g$
- After defining the function $g$ and in order to devise an iterative algorithm to solve the equation $x=g(x)$ we consider an initial guess $x_0\approx x^\ast$
- We hope that this method will converge faster than the bisection method
- A good initial condition could be the result of the bisection method

### The fixed point algorithm

Suppose that $g(x)$ is given. Given an initial guess $x_0$, the **Fixed point iteration** is defined as
$$x_{k+1}=g(x_k), \quad k=0,1,2,\ldots$$
which a sequence of approximation $x_0,x_1,x_2,\cdots$

1. Set a tolerance $TOL$ for the accuracy
2. Set $k=0$
3. Initialize $x_k$
4. Set $Error=TOL+1$
5. while $Error>TOL$ do
  - Compute $x_{k+1}=g(x_k)$
  - Set $Error=|x_{k+1}-x_k|$
  - Increase the counter $k=k+1$
 6. end while
 7. Return the approximation of the root $x^\ast$


In [5]:
def fixedpoint(g, x0, tol = 1.e-6, maxit = 100):
    # g = the function g(x)
    # x0 = the initial guess of the fixed point x=g(x)
    # tol = tolerance for the absolute error
    #         of two subsequent approximations
    # maxit = maximum number of iterations allowed
    error = 1.0
    iteration = 0
    xk = x0
    while (error > tol and iteration < maxit):
        iteration = iteration + 1
        error = xk
        xk = g(xk)
        error = np.abs(error - xk)
        print ('iteration =', iteration, ', x =', xk)
    return xk

First we try the code for the function $f(x)=x^2-x-1$ using $g(x)=\sqrt{x+1}$.

In [6]:
def f(x):
    y = x**2-x-1.0
    return y
def g(x):
    y = np.sqrt(x+1.0)
    return y
tol = 1.e-4
maxit = 50
x0 = 0.0
x = fixedpoint(g, x0, tol, maxit)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

iteration = 1 , x = 1.0
iteration = 2 , x = 1.4142135623730951
iteration = 3 , x = 1.5537739740300374
iteration = 4 , x = 1.5980531824786175
iteration = 5 , x = 1.6118477541252516
iteration = 6 , x = 1.616121206508117
iteration = 7 , x = 1.6174427985273905
iteration = 8 , x = 1.617851290609675
iteration = 9 , x = 1.6179775309347393
iteration = 10 , x = 1.6180165422314876
The approximate solution x is:  1.6180165422314876
And the value f(x) is:  -3.9011296748103774e-05


We try to solve the same equation $f(x)=0$ but now with $g(x)=1+1/x$.

In [7]:
def g(x):
    y = 1.0+1.0/x
    return y
tol = 1.e-4
maxit = 50
x0 = 1.0
x = fixedpoint(g, x0, tol, maxit)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

iteration = 1 , x = 2.0
iteration = 2 , x = 1.5
iteration = 3 , x = 1.6666666666666665
iteration = 4 , x = 1.6
iteration = 5 , x = 1.625
iteration = 6 , x = 1.6153846153846154
iteration = 7 , x = 1.619047619047619
iteration = 8 , x = 1.6176470588235294
iteration = 9 , x = 1.6181818181818182
iteration = 10 , x = 1.6179775280898876
iteration = 11 , x = 1.6180555555555556
The approximate solution x is:  1.6180555555555556
And the value f(x) is:  4.822530864223573e-05


Try using the functions
$$g(x)=x^2-1$$
and
$$g(x)=x-\frac{x^2-x-1}{2x-1}$$

## Newton's Method (Newton-Raphson)
 
**Basic Idea**: Given $f(x)$ and $f'(x)$ and and initial guess $x_0$, find the root of the tangent line to $(x_0,f(x_0))$ to find $x_1\approx x^*$. Continue using $x_1$ to compute $x_2$, etc.

Given $x_k$, then the next approximation of the root $x^\ast$ defined by Newton's method is:
$$
    x_{k+1} = x_k-\frac{f(x_k)}{f'(x_k)}
$$

### Newton's method algorith 

1. Set a tolerance $TOL$ for the accuracy
2. Set the maximum number of iteration $MAXIT$ 
3. Set $k=0$
4. Initialize $x_k$ and $Error=TOL+1$
5. while $Error>TOL$ and $k<MAXIT$ do
    - if $f'(x_k)\not=0$ then
        - Compute $x_{k+1}=x_k-\frac{f(x_k)}{f'(x_{k})}$
        - Set $Error = |x_{k+1}-x_k|$
        - Increase the counter $k=k+1$
    - end if
6. end while
7. Return the approximation of the root $x^\ast$

In [8]:
def newton(f, df, x0, tol = 1.e-6, maxit = 100):
    # f = the function f(x)
    # df = the derivative of f(x)
    # x0 = the initial guess of the solution
    # tol = tolerance for the absolute error
    # maxit = maximum number of iterations
    err = tol+1.0
    iteration = 0
    xk = x0
    while (err > tol and iteration < maxit):
        iteration = iteration + 1
        err = xk # store previous approximation to err
        xk = xk - f(xk)/df(xk) # Newton's iteration
        err = np.abs(err - xk) # compute the new error
        print(iteration, xk)
    return xk

We try our method for the equation
$$ \ln x + x = 0 $$

Note that here we don't need to specify an initial interval but an initial guess, which we take to be $x_0=1$. 

In this case $f(x)=\ln x + x$ and $f'(x)=\frac{1}{x} + 1$.

In [9]:
def f(x):
    y = np.log(x) + x
    return y
def df(x):
    y = 1.0 / x + 1.0
    return y
tol = 1.e-4
maxit = 50
x0 = 1.0
x = newton(f, df, x0, tol, maxit)
print('The aproximate solution is: ', x)
print('And the error is: ', f(x))

1 0.5
2 0.5643823935199818
3 0.5671389877150601
4 0.5671432903993691
The aproximate solution is:  0.5671432903993691
And the error is:  -2.877842408821607e-11


The rate of convergence for Newton's method is $r=2$.

## Secant Method

The secant method can be obtained from Newton's method with the approximation of the first derivative
$$f'(x_k)=\frac{f(x_k)-f(x_{k-1})}{x_k-x_{k-1}}$$

The new iteration is the defined
Given $x_k$ and $x_{k-1}
$$x_{k+1}=x_k-\frac{x_k-x_{k-1}}{f(x_k)-f(x_{k-1})}f(x_k)$$

### Secant method algorithm 

1. Set a tolerance $TOL$ for the accuracy
2. Set the maximum number of iteration $MAXIT$ 
4. Initialize $x_{k-1}$ and $x_k$ and set $k=0$
5. Set $Error=TOL+1$
6. while $Error>TOL$ and $k<MAXIT$ do
    - if $f'(x_k)\not=0$ then
        - Compute $x_{k+1}=x_k-\frac{x_k-x_{k-1}}{f(x_k)-f(x_{k-1})}f(x_k)$
        - Set $Error = |x_{k+1}-x_k|$
        - Increase the counter $k=k+1$
    - end if
7. end while
8. Return the approximation of the root $x^\ast$

In [10]:
def secant(f, x1, x2, tol = 1.e-6, maxit = 100):
    # f = the function f(x)
    # x1 = an initial guess of the solution
    # x2 = another initial guess of the solution
    # tol = tolerance for the absolute error
    # maxit = maximum number of iterations
    err = 1.0
    iteration = 0
    while (err > tol and iteration < maxit):
        xk1 = x1
        xk = x2
        iteration = iteration + 1
        err = xk1
        xk1 = xk - (xk-xk1)/(f(xk)-f(xk1))*f(xk)
        err = np.abs(err - xk1)
        x1 = x2
        x2 = xk1
        print(iteration, xk1)
    return xk1

Testing this new method for the equation
$$ \ln x + x = 0\ , $$
with initial guesses $x_{-1}=1$ and $x_{0}=2$ we obtain the following results: 

In [11]:
def f(x):
    y = np.log(x) + x
    return y
tol = 1.e-4
maxit = 50
x1 = 1.0
x2 = 2.0
x = secant(f, x1, x2, tol, maxit)
print('The approximate solution is: ', x)
print('And the error is: ', f(x))

1 0.40938389085035864
2 0.651575386390747
3 0.5751035382227284
4 0.5667851889083253
5 0.5671448866112347
6 0.5671432907314143
7 0.5671432904097836
The approximate solution is:  0.5671432904097836
And the error is:  -6.661338147750939e-16


The rate of convergence for the secant method is the golder ratio $r=\frac{1+\sqrt{5}}{2}\approx 1.61<2$ and thus is a bit slower than Newton's method.

## The Module `scipy.optimize`

The bisection method is implemented in the function `bisect` of the module `scipy.optimize`

In [12]:
import scipy.optimize as spo
def f(x):
    y = np.log(x)+x
    return y
a = 0.1
b = 1.0
tol = 1.e-4
x = spo.bisect(f, a, b, () , tol)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

The approximate solution x is:  0.567193603515625
And the value f(x) is:  0.0001390223881425623


The generaic fixed-point method is also implemented in `scipy.optimize` in the function `fixed_point`

In [13]:
import scipy.optimize as spo
def f(x):
    y = x**2-x-1.0
    return y
def g(x):
    y = np.sqrt(x+1.0)
    return y
x0 = 1.0
tol = 1.e-4
maxit = 50
x = spo.fixed_point(g, x0, (), tol, maxit)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

The approximate solution x is:  1.6180339887498991
And the value f(x) is:  9.547918011776346e-15


Both the Newton and Secant methods are implemented in the function `newton` of `scipy.optimize`.

In [14]:
import scipy.optimize as spo
def f(x):
    y = np.log(x)+x
    return y
def df(x):
    y = 1.0/x+1.0
    return y
x0 = 1.0
x = spo.newton(f, x0, df, tol=1.e-4, maxiter=50)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

The approximate solution x is:  0.5671432903993691
And the value f(x) is:  -2.877842408821607e-11


In `scipy.optimize` one can find also a hybrid method that works in a more broader spectrum of problems compared to the previous implementation. This method is implemented in the function `fsolve`.

In [15]:
import scipy.optimize as spo
def f(x):
    y = np.log(x)+x
    return y
def df(x):
    y = 1.0/x+1.0
    return y
x0 = 1.0
x = spo.fsolve(f, x0, fprime=df, xtol=1.e-4)
print('The approximate solution x is: ', x)
print('And the value f(x) is: ', f(x))

The approximate solution x is:  [0.56714329]
And the value f(x) is:  [3.4803842e-09]


## Application in Astrophysics

If $\psi$ is the mean anomaly of the orbit of a plant, then $\theta$, the eccentric anomaly, can be computed by solving the fixed point equation
$$\theta=\psi+e\sin \theta$$
where $e$ is the eccentricity of the elliptical orbit.

This equation can be solved seamlesly using the funtion `fixed_point` of `scipy.optimize`.

In [16]:
import numpy as np
import scipy.optimize as spo
def g(theta):
    e = 1.e-6
    psi = np.pi/6.0
    return psi+e*np.sin(theta)
theta0 = np.pi/6.0
theta = spo.fixed_point(g, theta0)
print('eccentric anomaly=', theta)

eccentric anomaly= 0.5235992755987319


Knowing the eccentric anomaly can lead to the estimation of the heliocentric distance $r=a(1-e\cos\theta)$ where $a$ is the semi-major axis.